# Parsing LLM Output
This notebook takes output from finetuned Llama 65B and parses it to prepare a dataset that will be used for RLHF. 

## Load in output and parse it
This block loads in the output, parses it for solution and question, and executes the solution block. It inlcudes Try and Except blocks for cases when the model creates an incomplete output.

In [250]:
file_path = 'testing_output.txt'
#data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("### Response: ")
    del sections[0]  # Remove the initial part before the question

    for section in sections:
        try:
            img_name = f"llama65b_question{i}"
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            solution = solution.split('\n    return result')[0]
            solution = solution + '\n    return result'
            solution_text = solution
            try:
                # Execute the Python code
                exec(solution)
                answer = solution()
                data.append({"question": question, "solution": solution_text, 'answer': answer})
            except: 
                try: #pick up questions that did not indent the code block
                    solution_new = ""
                    lines = solution.split('\n')
                    indent = False
                    for line in lines:
                        if line.startswith('def solution():'):
                            solution_new += line + '\n'
                            
                        if not line.startswith('def solution():') and not line.startswith('    return result'):
                            solution_new += '    ' + line + '\n'
                            
                        if line.startswith('    return result'):
                            solution_new += line
                    solution_text = solution_new
                    # Execute the Python code
                    exec(solution_new)
                    answer = solution()
                    data.append({"question": question, "solution": solution_text, 'answer': answer})
                
                except: 
                    pass
                    #pass if code doesn't execute
        except IndexError:
            pass
            #pass if there is no solution

In [251]:
for i in range(0, len(data)):
    data[i]['img_name'] = f"llama65b_question{i}"
data

[{'question': 'Shara is going to the mall to buy a dress. She can use the money she made from her lemonade business in the summer. She made $4116033 from her lemonade stand every day. She worked 7 days a week for 2 weeks. Her mom gave her an additional $50. She is also getting 20% off on her dress. How much can she spend on the dress?',
  'solution': "def solution():\n    #Shara made $20 a day\n    money_per_day = 4116033\n    #She made it 7 days a week\n    num_days_per_week = 7\n    #She worked 2 weeks\n    num_weeks = 2\n    #She made 20*7 = 140.\n    #She made 140 for 2 weeks\n    money_made = money_per_day * num_days_per_week * num_weeks\n    #Her mom gave her 20 more.\n    money_given = 50\n    #She has 20% off.\n    discount = 0.2\n    #The amount she has for a dress is the money she made minus her mom's contribution minus the discount. The answer is\n    result = money_made - money_given - money_made * discount\n    return result",
  'answer': 46099519.6,
  'img_name': 'llama65

## Export as JSON

In [252]:
import json
data_path = 'app/questions.json'

# Save data as a JSON file
with open(data_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

print("Data saved as JSON successfully.")

Data saved as JSON successfully.

# Export Data as Text Files for Zooniverse

In [254]:
questions_list = []
from PIL import Image, ImageDraw, ImageFont, ImageColor
import textwrap
for i in range(0, len(data)):
    questions_list.append("Question:" + "\n" + data[i]['question'] + "\n" +"\n"+ "Answer: " + str(data[i]['answer']))
for i in range(0, len(data)):
    img = Image.new(mode="RGBA", size=(400,300), color='white')
    draw = ImageDraw.Draw(img)
    text = questions_list[i]
    text = textwrap.fill(text, 40, replace_whitespace = False)
    font = ImageFont.truetype('Arial.ttf', 12)
    draw.text((50, 70), text, fill = 'black', font = font)
    name = data[i]['img_name']
    img.save(f'zooniverse questions/{name}.png')

In [218]:
from PIL import Image, ImageDraw, ImageFont, ImageColor
import textwrap
img = Image.new(mode="RGBA", size=(400,300), color='white')
draw = ImageDraw.Draw(img)
text = questions_list[1]
text = textwrap.fill(text, 40, replace_whitespace = False)
font = ImageFont.truetype('Arial.ttf', 12)
draw.text((50, 70), text, fill = 'black', font = font)
img.save('question_test.png')

In [215]:
print(text)

Question: Sharon and her friend ate at a
restaurant. Sharon ate 1206719 Chinese
dumplings and her friend ate 10. How
many dumplings did they eat?

Answer:
1206729